In [30]:
import cv2 as cv
import numpy as np
import os
import torch
from pathlib import Path
import sys
sys.path.append('../unet')
from data_loader import ImgDataSet
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
from build_unet import load_unet_vgg16
from inference import evaluate_img

In [17]:
data_dir = '/home/wj/dataset/seg_dataset'
DIR_IMG  = os.path.join(data_dir, 'images')
DIR_MASK = os.path.join(data_dir, 'masks')
img_names  = [path.name for path in Path(DIR_IMG).glob('*.jpg')]
mask_names = [path.name for path in Path(DIR_MASK).glob('*.jpg')]


channel_means = [0.485, 0.456, 0.406]
channel_stds  = [0.229, 0.224, 0.225]
train_tfms = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(channel_means, channel_stds)])

val_tfms = transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize(channel_means, channel_stds)])

mask_tfms = transforms.Compose([transforms.ToTensor()])

dataset = ImgDataSet(img_dir=DIR_IMG, img_fnames=img_names, img_transform=train_tfms, mask_dir=DIR_MASK, mask_fnames=mask_names, mask_transform=mask_tfms)
train_size = int(0.01*len(dataset))
# train_dataset = RandomSampler(dataset, False, train_size)
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, 4, shuffle=False, pin_memory=torch.cuda.is_available(), num_workers=4)
# valid_loader = DataLoader(valid_dataset, args.batch_size, shuffle=False, pin_memory=torch.cuda.is_available(), num_workers=args.num_workers)

In [12]:
model = load_unet_vgg16('../models/model_unet_vgg_16_best.pt')

In [26]:
model.eval()

UNet16(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15)

In [50]:
import matplotlib.pyplot as plt
for i, (input, target) in enumerate(train_loader):
    if i == 1:
        break
    input_var  = Variable(input).cuda()
    pred = model(input_var)
    y = target[0,0].numpy()
    y_true = F.sigmoid(target[0, 0]).numpy()
    mask = F.sigmoid(pred[3, 0]).data.cpu().numpy()
    print(input.shape)
    print(pred[3,0])
    # print(mask.shape)
    # plt.imshow(mask, cmap='gray')
    

torch.Size([4, 3, 448, 448])
tensor([[-2.4519, -3.6856, -4.0472,  ..., -4.2260, -3.5156, -2.3451],
        [-3.6934, -5.3575, -5.8961,  ..., -5.6299, -5.2005, -3.3800],
        [-4.2193, -5.9509, -6.4521,  ..., -6.0564, -5.3836, -3.7280],
        ...,
        [-4.4144, -6.3091, -6.8834,  ..., -8.5258, -7.6760, -4.9599],
        [-4.2937, -6.1815, -7.0508,  ..., -7.6960, -6.9337, -4.3815],
        [-3.0752, -4.5021, -5.1519,  ..., -5.0219, -4.5308, -3.0201]],
       device='cuda:0', grad_fn=<SelectBackward0>)


In [19]:
input, target = train_loader[0]
input

TypeError: 'DataLoader' object is not subscriptable

In [2]:
import sys
sys.path.append("/home/wj/local/crack_segmentation/DeepCrack/codes")
from model.deepcrack import DeepCrack
model = DeepCrack()
model.eval()

DeepCrack(
  (down1): Down(
    (nn): Sequential(
      (0): ConvRelu(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU(inplace=True)
      )
      (1): ConvRelu(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU(inplace=True)
      )
    )
    (maxpool_with_argmax): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down2): Down(
    (nn): Sequential(
      (0): ConvRelu(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU(inplace=True)
      )
      (1): ConvRelu(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU(inplace=True)
      )
    )
    (maxpool_with_argmax): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down3): Down(
    (nn): Sequential(
      (0): ConvRelu(
        (conv): C

In [9]:

import numpy as np
import torch
# input = np.random.rand(512, 512)
mask = np.random.rand(10, 512, 512)
# input = torch.rand(10, 512, 512)
# mask = torch.rand(10, 512, 512)
inputs=[]
for i in range(10):
    input = np.random.rand(512, 512)
    inputs.append(input)
inputs = np.array(inputs)

In [13]:
print(inputs.shape)

(10, 512, 512)


In [10]:
inputs[inputs>0.5] = 1

In [11]:
inputs[inputs<=0.5] = 0
np.sum(inputs == 1)

1311238

In [12]:
np.sum(inputs == 0)

1310202